# Actividad Grupal: Análisis de Desigualdad - Versión con Problemas de Visualización

Este notebook contiene **visualizaciones problemáticas** que demuestran cómo NO se deben crear gráficos para análisis de datos. Los problemas incluyen:

1. **Escalas mal ajustadas**: Ejes Y truncados que exageran falsamente las diferencias
2. **Paletas de colores inadecuadas**: Colores similares que dificultan la comparación entre países


In [215]:
import pandas as pd

# La URL "raw" es necesaria para que pandas pueda leer el flujo de datos directamente
ruta_github = "https://github.com/HugoAguilarReyna/Visualizacion_Inter_Info/raw/454eaf147621d033b7c2843363266a6aaf701cd2/API_SI.POV.GINI_DS2_es_excel_v2_126621.xls"

try:
    # Para archivos .xls antiguos, a veces es necesario especificar el motor 'xlrd'
    df = pd.read_excel(ruta_github, sheet_name='Data')
    print("¡Datos cargados con éxito desde GitHub!")
    print(df.head()) # Verificamos los primeros registros
except Exception as e:
    print(f"Error al cargar el archivo: {e}")
    print("\nNota: Asegúrate de tener instalado xlrd (pip install xlrd) para leer archivos .xls")


¡Datos cargados con éxito desde GitHub!
  Country Name Country Code  Indicator Name Indicator Code  1960  1961  1962  \
0        Aruba          ABW  Índice de Gini    SI.POV.GINI   NaN   NaN   NaN   
1          NaN          AFE  Índice de Gini    SI.POV.GINI   NaN   NaN   NaN   
2   Afganistán          AFG  Índice de Gini    SI.POV.GINI   NaN   NaN   NaN   
3          NaN          AFW  Índice de Gini    SI.POV.GINI   NaN   NaN   NaN   
4       Angola          AGO  Índice de Gini    SI.POV.GINI   NaN   NaN   NaN   

   1963  1964  1965  ...  2015  2016  2017  2018  2019  2020  2021  2022  \
0   NaN   NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
1   NaN   NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
2   NaN   NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
3   NaN   NaN   NaN  ...   NaN   NaN   NaN   NaN   NaN   NaN   NaN   NaN   
4   NaN   NaN   NaN  ...   NaN   NaN   NaN  51.3   NaN   NaN   NaN   NaN   

   2023  2024  
0   Na

In [216]:
# Las columnas que NO contienen los años (variables de identificación)
id_vars = ['Country Name', 'Country Code', 'Indicator Name', 'Indicator Code']

# Las columnas que SÍ contienen los años (variables a "derretir")
year_cols = df.columns.difference(id_vars).tolist()

# 1. Aplicar melt()
df_long = df.melt(
    id_vars=id_vars,
    value_vars=year_cols,
    var_name='Year',
    value_name='Value'
)

# 2. Convertir la columna 'Year' a tipo numérico (entero) para análisis
df_long['Year'] = pd.to_numeric(df_long['Year'])

# 3. Mostrar las primeras filas del nuevo DataFrame
print("Primeras filas del DataFrame en formato largo:")
print(df_long.head())


Primeras filas del DataFrame en formato largo:
  Country Name Country Code  Indicator Name Indicator Code  Year  Value
0        Aruba          ABW  Índice de Gini    SI.POV.GINI  1960    NaN
1          NaN          AFE  Índice de Gini    SI.POV.GINI  1960    NaN
2   Afganistán          AFG  Índice de Gini    SI.POV.GINI  1960    NaN
3          NaN          AFW  Índice de Gini    SI.POV.GINI  1960    NaN
4       Angola          AGO  Índice de Gini    SI.POV.GINI  1960    NaN


In [217]:
df_clean = df_long.dropna(subset=['Value'])
print(f"Filas eliminadas (NaN en Value): {len(df_long) - len(df_clean)}")
df_clean = df_clean.dropna(subset=['Country Name'])


Filas eliminadas (NaN en Value): 14888


In [218]:
# Lista de agregaciones a excluir
names_to_exclude = [
    'Europa Central y del Báltico', 'El mundo árabe', 'Asia oriental y el Pacífico (excluido altos ingresos)',
    'inicial del dividendo demográfico', 'Asia oriental y el Pacífico',
    'Europa y Asia central (excluido altos ingresos)', 'Europa y Asia central',
    'Zona del Euro', 'Unión Europea', 'Frágiles y situaciones de conflicto afectados',
    'Islas Feroe', 'Gibraltar', 'Groenlandia', 'Guam', 'Ingreso alto',
    'Países pobres muy endeudados (PPME)', 'Sólo BIRF', 'BIRF y la AIF',
    'total de la AIF', 'mezcla de la AIF', 'Sólo AIF', 'No clasificado',
    'América Latina y el Caribe (excluido altos ingresos)', 'América Latina y el Caribe',
    'Países menos desarrollados: clasificación de las Naciones Unidas',
    'Países de ingreso bajo', 'Países de ingreso mediano bajo', 'Ingreso mediano y bajo',
    'avanzada del dividendo demográfico', 'Oriente Medio, Norte de África, Afganistán y Pakistán',
    'Ingreso mediano', 'Oriente Medio y Norte de África (excluido altos ingresos)',
    'América del Norte', 'Miembros OCDE', 'Otros Estados pequeos',
    'previa al dividendo demográfico', 'Puerto Rico (US)', 'Ribera Occidental y Gaza',
    'Estados pequeos de las Islas del Pacfico', 'posterior al dividendo demográfico',
    'Polinesia Francesa', 'Asia meridional', 'África al sur del Sahara (excluido altos ingresos)',
    'Sudán del Sur', 'África al sur del Sahara', 'Pequeños Estados', 'República Árabe Siria',
    'Islas Turcas y Caicos', 'Asia oriental y el Pacífico (BIRF y la AIF)',
    'Europa y Asia central (BIRF y la AIF)', 'América Latina y el Caribe (BIRF y la AIF)',
    'Oriente Medio y Norte de África (BIRF y la AIF)', 'Asia meridional (BIRF y la AIF)',
    'África al sur del Sahara (BIRF y la AIF)', 'Ingreso mediano alto',
    'Islas Vírgenes Británicas', 'Islas Vírgenes (EE.UU.)', 'Mundo', 'Kosovo'
]

df_countries_by_name = df_long[~df_long['Country Name'].isin(names_to_exclude)].dropna(subset=['Value'])

print(f"Filas originales con datos válidos: {len(df_long.dropna(subset=['Value']))}")
print(f"Filas después de la eliminación de agregaciones: {len(df_countries_by_name)}")


Filas originales con datos válidos: 2402
Filas después de la eliminación de agregaciones: 2369


In [219]:
# Lista de países europeos
PAISES_EUROPEOS = [
    'Albania', 'Alemania', 'Armenia', 'Austria', 'Azerbaiyán', 'Belarús', 
    'Bélgica', 'Bosnia y Herzegovina', 'Bulgaria', 'Chipre', 'Croacia', 
    'Dinamarca', 'Eslovenia', 'España', 'Estonia', 'Federación de Rusia', 
    'Finlandia', 'Francia', 'Georgia', 'Grecia', 'Hungría', 'Irlanda', 
    'Islandia', 'Italia', 'Kazajstán', 'Letonia', 'Lituania', 'Luxemburgo', 
    'Macedonia del Norte', 'Malta', 'Montenegro', 'Noruega', 'Países Bajos', 
    'Polonia', 'Portugal', 'Reino Unido', 'República Checa', 'República de Moldova', 
    'República Eslovaca', 'Rumania', 'Serbia', 'Suecia', 'Suiza', 'Turquía', 
    'Ucrania'
]

def get_agrupacion(country):
    occidental_nordica = [
        'Alemania', 'Austria', 'Bélgica', 'Dinamarca', 'Eslovenia', 'España', 
        'Finlandia', 'Francia', 'Grecia', 'Islandia', 'Irlanda', 'Italia', 
        'Luxemburgo', 'Malta', 'Noruega', 'Países Bajos', 'Portugal', 
        'Reino Unido', 'Suecia', 'Suiza'
    ]
    oriental_central = [
        'Albania', 'Belarús', 'Bosnia y Herzegovina', 'Bulgaria', 'Chipre', 
        'Croacia', 'Estonia', 'Hungría', 'Letonia', 'Lituania', 'Macedonia del Norte', 
        'Montenegro', 'Polonia', 'República Checa', 'República de Moldova', 
        'República Eslovaca', 'Rumania', 'Serbia', 'Ucrania'
    ]
    
    if country in occidental_nordica:
        return 'Europa Occidental / Nórdica'
    elif country in oriental_central:
        return 'Europa Oriental / Central'
    else: 
        return 'Transcontinental / Cáucaso'

df_countries_by_name['Agrupacion'] = df_countries_by_name['Country Name'].apply(get_agrupacion)
df_paises_europeos = df_countries_by_name[df_countries_by_name['Country Name'].isin(PAISES_EUROPEOS)]


## Parte 1: Visualización con Problemas

### Problema 1: Escalas mal ajustadas y paleta de colores inadecuada

Esta visualización demuestra los problemas mencionados:
- **Eje Y truncado**: No comienza en 0, exagerando falsamente las diferencias
- **Paleta de colores problemática**: Colores muy similares que dificultan la comparación


In [220]:
import plotly.express as px
import plotly.graph_objects as go
import pandas as pd

# Seleccionar algunos países para la visualización problemática
PAISES_FILTRADOS = ['Reino Unido', 'Francia', 'Italia', 'Noruega', 'Suecia', 'Alemania', 'España', 'Polonia']

df_estudio = df_paises_europeos[df_paises_europeos['Country Name'].isin(PAISES_FILTRADOS)].copy()

if df_estudio.empty:
    print("❌ Error: No se encontraron los países solicitados en el DataFrame.")
else:
    # ==========================================================
    # ⚠️ VISUALIZACIÓN PROBLEMÁTICA 1: Eje Y truncado
    # ==========================================================
    print("⚠️ PROBLEMA 1: Eje Y truncado (no comienza en 0)")
    print("   Esto exagera falsamente las diferencias entre países")
    
    # Calcular el rango mínimo y máximo de los datos
    gini_min = df_estudio['Value'].min()
    gini_max = df_estudio['Value'].max()
    
    # ⚠️ PROBLEMA: Truncar el eje Y para exagerar diferencias
    # En lugar de [0, 50], usamos un rango muy estrecho
    Y_MIN_PROBLEMA = gini_min - 1  # No comienza en 0
    Y_MAX_PROBLEMA = gini_max + 1  # Rango muy estrecho
    
    # ⚠️ PROBLEMA: Paleta de colores con tonos muy similares (grises)
    # Esto dificulta distinguir entre países
    COLOR_PALETTE_PROBLEMA = [
        '#808080',  # Gris medio
        '#888888',  # Gris ligeramente más claro
        '#909090',  # Gris más claro
        '#989898',  # Gris aún más claro
        '#A0A0A0',  # Gris claro
        '#A8A8A8',  # Gris muy claro
        '#B0B0B0',  # Gris casi blanco
        '#B8B8B8'   # Gris muy claro
    ]
    
    fig_problema = px.line(
        df_estudio.sort_values('Year'),
        x='Year',
        y='Value',
        color='Country Name',
        color_discrete_sequence=COLOR_PALETTE_PROBLEMA,  # Colores muy similares
        markers=True,
        title='<b>⚠️ PROBLEMÁTICO: Evolución del Gini con Escala Truncada y Colores Similares</b>',
        labels={'Value': 'Índice de Gini', 'Year': 'Año'}
    )
    
    # ⚠️ Configuración problemática del eje Y
    fig_problema.update_layout(
        template='plotly_dark',
        plot_bgcolor='black',
        paper_bgcolor='black',
        yaxis=dict(
            range=[Y_MIN_PROBLEMA, Y_MAX_PROBLEMA],  # ⚠️ Rango truncado, no desde 0
            gridcolor='#333'
        ),
        xaxis=dict(gridcolor='#333'),
        legend=dict(
            orientation="h",
            yanchor="top",
            y=-0.3,
            xanchor="center",
            x=0.5
        )
    )
    
    fig_problema.show()
    
    print(f"\n📊 Rango del eje Y problemático: [{Y_MIN_PROBLEMA:.1f}, {Y_MAX_PROBLEMA:.1f}]")
    print(f"   (Debería ser [0, 50] para una perspectiva real)")
    print(f"   ⚠️ Esto hace que diferencias pequeñas parezcan enormes!")


⚠️ PROBLEMA 1: Eje Y truncado (no comienza en 0)
   Esto exagera falsamente las diferencias entre países



📊 Rango del eje Y problemático: [21.9, 39.8]
   (Debería ser [0, 50] para una perspectiva real)
   ⚠️ Esto hace que diferencias pequeñas parezcan enormes!


In [221]:
# ==========================================================
# ⚠️ VISUALIZACIÓN PROBLEMÁTICA 2: Múltiples países con colores similares
# ==========================================================
print("\n⚠️ PROBLEMA 2: Múltiples países con colores muy similares")
print("   Esto hace imposible seguir las líneas de cada país")

# Seleccionar más países para empeorar el problema
PAISES_MUCHOS = ['Reino Unido', 'Francia', 'Italia', 'Noruega', 'Suecia', 
                 'Alemania', 'España', 'Polonia', 'Bélgica', 'Austria', 
                 'Dinamarca', 'Finlandia', 'Portugal', 'Grecia', 'Países Bajos']

df_muchos = df_paises_europeos[df_paises_europeos['Country Name'].isin(PAISES_MUCHOS)].copy()

# ⚠️ PROBLEMA: Paleta con colores muy similares (todos en tonos azules/grises)
COLOR_PALETTE_SIMILAR = [
    '#4A90E2', '#5A9FE2', '#6AAEE2', '#7ABDE2', '#8ACCE2',  # Azules muy similares
    '#9ADBE2', '#AAEAE2', '#BAF9E2', '#CAFFE2', '#DAFFE2',  # Azules-verdes similares
    '#E2E2E2', '#E8E8E8', '#EEEEEE', '#F4F4F4', '#FAFAFA'   # Grises casi idénticos
]

fig_muchos = px.line(
    df_muchos.sort_values('Year'),
    x='Year',
    y='Value',
    color='Country Name',
    color_discrete_sequence=COLOR_PALETTE_SIMILAR,  # ⚠️ Colores muy similares
    markers=True,
    title='<b>⚠️ PROBLEMÁTICO: 15 Países con Colores Casi Idénticos</b>',
    labels={'Value': 'Índice de Gini', 'Year': 'Año'}
)

# ⚠️ También truncar el eje Y para empeorar el problema
gini_min_muchos = df_muchos['Value'].min()
gini_max_muchos = df_muchos['Value'].max()

fig_muchos.update_layout(
    template='plotly_dark',
    plot_bgcolor='black',
    paper_bgcolor='black',
    yaxis=dict(
        range=[gini_min_muchos - 2, gini_max_muchos + 2],  # ⚠️ Truncado
        gridcolor='#333'
    ),
    xaxis=dict(gridcolor='#333'),
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02,
        font=dict(size=8)  # ⚠️ Leyenda pequeña y difícil de leer
    ),
    margin=dict(r=150)  # Espacio para leyenda
)

fig_muchos.show()

print(f"\n⚠️ Problemas identificados:")
print(f"   1. {len(PAISES_MUCHOS)} países con colores casi idénticos")
print(f"   2. Eje Y truncado: [{gini_min_muchos - 2:.1f}, {gini_max_muchos + 2:.1f}]")
print(f"   3. Leyenda pequeña y difícil de leer")
print(f"   4. Imposible distinguir qué línea corresponde a cada país")



⚠️ PROBLEMA 2: Múltiples países con colores muy similares
   Esto hace imposible seguir las líneas de cada país



⚠️ Problemas identificados:
   1. 15 países con colores casi idénticos
   2. Eje Y truncado: [20.2, 40.9]
   3. Leyenda pequeña y difícil de leer
   4. Imposible distinguir qué línea corresponde a cada país


In [222]:
# ==========================================================
# ⚠️ VISUALIZACIÓN PROBLEMÁTICA 3: Box Plot con escala truncada
# ==========================================================
print("\n⚠️ PROBLEMA 3: Box Plot con escala truncada")
print("   Esto exagera las diferencias entre países")

fig_box_problema = px.box(
    df_estudio,
    x='Country Name',
    y='Value',
    color='Country Name',
    points="all",
    color_discrete_sequence=COLOR_PALETTE_PROBLEMA,  # ⚠️ Colores similares
    title='<b>⚠️ PROBLEMÁTICO: Box Plot con Escala Truncada</b>'
)

# ⚠️ PROBLEMA: Eje Y truncado
gini_min_box = df_estudio['Value'].min()
gini_max_box = df_estudio['Value'].max()

fig_box_problema.update_layout(
    template='plotly_dark',
    plot_bgcolor='black',
    paper_bgcolor='black',
    yaxis=dict(
        range=[gini_min_box - 1, gini_max_box + 1],  # ⚠️ No comienza en 0
        gridcolor='#333'
    ),
    showlegend=False,
    xaxis=dict(tickangle=45)
)

fig_box_problema.show()

print(f"\n⚠️ Rango del eje Y: [{gini_min_box - 1:.1f}, {gini_max_box + 1:.1f}]")
print(f"   Debería ser [0, 50] para mostrar la verdadera magnitud de las diferencias")



⚠️ PROBLEMA 3: Box Plot con escala truncada
   Esto exagera las diferencias entre países



⚠️ Rango del eje Y: [21.9, 39.8]
   Debería ser [0, 50] para mostrar la verdadera magnitud de las diferencias


## Comparación: Visualización Correcta vs. Problemática

A continuación se muestra una comparación lado a lado para demostrar el impacto de estos problemas.


In [223]:
# ==========================================================
# COMPARACIÓN: Correcta vs. Problemática
# ==========================================================

# VISUALIZACIÓN CORRECTA
print("✅ VISUALIZACIÓN CORRECTA:")
print("   - Eje Y desde 0 hasta 50")
print("   - Paleta de colores contrastantes y distinguibles")

COLOR_PALETTE_CORRECTA = ['#FF00FF', '#00FFFF', '#00FF00', '#FF4500', '#9400D3',
                          '#FF1493', '#00CED1', '#32CD32']

fig_correcta = px.line(
    df_estudio.sort_values('Year'),
    x='Year',
    y='Value',
    color='Country Name',
    color_discrete_sequence=COLOR_PALETTE_CORRECTA,  # ✅ Colores contrastantes
    markers=True,
    title='<b>✅ CORRECTO: Evolución del Gini con Escala Completa y Colores Distinguibles</b>',
    labels={'Value': 'Índice de Gini', 'Year': 'Año'}
)

fig_correcta.update_layout(
    template='plotly_dark',
    plot_bgcolor='black',
    paper_bgcolor='black',
    yaxis=dict(
        range=[0, 50],  # ✅ Comienza en 0, rango completo
        gridcolor='#333'
    ),
    xaxis=dict(gridcolor='#333'),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.3,
        xanchor="center",
        x=0.5
    )
)

fig_correcta.show()

print("\n" + "="*80)
print("RESUMEN DE PROBLEMAS IDENTIFICADOS:")
print("="*80)
print("1. ❌ Eje Y truncado: Exagera falsamente las diferencias")
print("2. ❌ Paleta de colores similar: Dificulta distinguir países")
print("3. ❌ Múltiples países con colores casi idénticos: Imposible seguir líneas")
print("4. ❌ Leyenda mal posicionada o muy pequeña: Dificulta la lectura")
print("\n✅ SOLUCIONES:")
print("1. ✅ Eje Y desde 0: Muestra la verdadera magnitud")
print("2. ✅ Paleta contrastante: Facilita la comparación")
print("3. ✅ Limitar número de países o usar small multiples")
print("4. ✅ Leyenda clara y bien posicionada")


✅ VISUALIZACIÓN CORRECTA:
   - Eje Y desde 0 hasta 50
   - Paleta de colores contrastantes y distinguibles



RESUMEN DE PROBLEMAS IDENTIFICADOS:
1. ❌ Eje Y truncado: Exagera falsamente las diferencias
2. ❌ Paleta de colores similar: Dificulta distinguir países
3. ❌ Múltiples países con colores casi idénticos: Imposible seguir líneas
4. ❌ Leyenda mal posicionada o muy pequeña: Dificulta la lectura

✅ SOLUCIONES:
1. ✅ Eje Y desde 0: Muestra la verdadera magnitud
2. ✅ Paleta contrastante: Facilita la comparación
3. ✅ Limitar número de países o usar small multiples
4. ✅ Leyenda clara y bien posicionada


## Parte 2: Interpretación de Gini en diferentes contextos

### Ejercicio 1: Países con evolución similar vs. divergente

**Objetivo**: Identificar países con evolución similar o divergente en términos de desigualdad.

**Problema a identificar**: Gráfico con líneas superpuestas y colores similares, dificultando el seguimiento de cada país.


In [224]:
# ==========================================================
# ⚠️ EJERCICIO 1 - PROBLEMA: Líneas superpuestas con colores similares
# ==========================================================
print("⚠️ EJERCICIO 1: Problema de líneas superpuestas y colores similares")
print("   Esto dificulta identificar países con evolución similar vs. divergente")

# Seleccionar países que queremos comparar (algunos con evolución similar, otros divergente)
PAISES_COMPARACION = ['Reino Unido', 'Francia', 'Italia', 'Noruega', 'Suecia', 
                      'Alemania', 'España', 'Polonia', 'Bélgica', 'Austria']

df_comparacion = df_paises_europeos[df_paises_europeos['Country Name'].isin(PAISES_COMPARACION)].copy()

# ⚠️ PROBLEMA: Colores muy similares en tonos pastel que se confunden
COLOR_PALETTE_SUPERPUESTA = [
    '#FFB6C1',  # Rosa claro
    '#FFC0CB',  # Rosa
    '#FFDAB9',  # Melocotón claro
    '#FFE4B5',  # Mocasín
    '#FFF8DC',  # Beige
    '#F0E68C',  # Caqui
    '#E6E6FA',  # Lavanda
    '#DDA0DD',  # Ciruela
    '#D8BFD8',  # Cardo
    '#DA70D6'   # Orquídea
]

fig_superpuesta = px.line(
    df_comparacion.sort_values('Year'),
    x='Year',
    y='Value',
    color='Country Name',
    color_discrete_sequence=COLOR_PALETTE_SUPERPUESTA,  # ⚠️ Colores muy similares
    markers=True,
    title='<b>⚠️ PROBLEMÁTICO: Líneas Superpuestas con Colores Similares</b><br><sub>Imposible distinguir qué país tiene evolución similar o divergente</sub>',
    labels={'Value': 'Índice de Gini', 'Year': 'Año'}
)

# ⚠️ También truncar el eje Y
gini_min_comp = df_comparacion['Value'].min()
gini_max_comp = df_comparacion['Value'].max()

fig_superpuesta.update_layout(
    template='plotly_dark',
    plot_bgcolor='black',
    paper_bgcolor='black',
    yaxis=dict(
        range=[gini_min_comp - 1, gini_max_comp + 1],  # ⚠️ Truncado
        gridcolor='#333'
    ),
    xaxis=dict(gridcolor='#333'),
    legend=dict(
        orientation="v",
        yanchor="top",
        y=1,
        xanchor="left",
        x=1.02,
        font=dict(size=9)  # ⚠️ Leyenda pequeña
    ),
    margin=dict(r=150)
)

fig_superpuesta.show()

print(f"\n⚠️ Problemas identificados:")
print(f"   1. {len(PAISES_COMPARACION)} países con colores pastel muy similares")
print(f"   2. Líneas completamente superpuestas - imposible seguir cada país")
print(f"   3. No se puede identificar qué países tienen evolución similar")
print(f"   4. No se puede identificar qué países tienen evolución divergente")
print(f"   5. Eje Y truncado: [{gini_min_comp - 1:.1f}, {gini_max_comp + 1:.1f}]")


⚠️ EJERCICIO 1: Problema de líneas superpuestas y colores similares
   Esto dificulta identificar países con evolución similar vs. divergente



⚠️ Problemas identificados:
   1. 10 países con colores pastel muy similares
   2. Líneas completamente superpuestas - imposible seguir cada país
   3. No se puede identificar qué países tienen evolución similar
   4. No se puede identificar qué países tienen evolución divergente
   5. Eje Y truncado: [21.9, 39.8]


In [225]:
# ==========================================================
# ✅ EJERCICIO 1 - SOLUCIÓN: Visualización que resalta similitudes y diferencias
# ==========================================================
print("\n✅ SOLUCIÓN: Visualización que permite identificar similitudes y diferencias")

# Agrupar países por evolución similar (basado en correlación de tendencias)
# Países con evolución similar: Noruega, Suecia, Dinamarca (nórdicos)
# Países con evolución divergente: Reino Unido (aumenta), Francia (disminuye)

PAISES_SIMILARES = ['Noruega', 'Suecia', 'Dinamarca', 'Finlandia']  # Nórdicos - evolución similar
PAISES_DIVERGENTES = ['Reino Unido', 'Francia', 'Italia']  # Evolución diferente

df_similares = df_paises_europeos[df_paises_europeos['Country Name'].isin(PAISES_SIMILARES)].copy()
df_divergentes = df_paises_europeos[df_paises_europeos['Country Name'].isin(PAISES_DIVERGENTES)].copy()

# Colores contrastantes para cada grupo
COLOR_SIMILARES = ['#00FF00', '#32CD32', '#7CFC00', '#90EE90']  # Verdes para similares
COLOR_DIVERGENTES = ['#FF00FF', '#FF1493', '#FF69B4']  # Magentas/Rosas para divergentes

fig_solucion = go.Figure()

# Agregar países con evolución similar (grupo 1)
for idx, pais in enumerate(PAISES_SIMILARES):
    df_pais = df_similares[df_similares['Country Name'] == pais].sort_values('Year')
    if not df_pais.empty:
        fig_solucion.add_trace(go.Scatter(
            x=df_pais['Year'],
            y=df_pais['Value'],
            mode='lines+markers',
            name=f'{pais} (Similar)',
            line=dict(width=3, color=COLOR_SIMILARES[idx % len(COLOR_SIMILARES)], dash='solid'),
            marker=dict(size=6)
        ))

# Agregar países con evolución divergente (grupo 2)
for idx, pais in enumerate(PAISES_DIVERGENTES):
    df_pais = df_divergentes[df_divergentes['Country Name'] == pais].sort_values('Year')
    if not df_pais.empty:
        fig_solucion.add_trace(go.Scatter(
            x=df_pais['Year'],
            y=df_pais['Value'],
            mode='lines+markers',
            name=f'{pais} (Divergente)',
            line=dict(width=3, color=COLOR_DIVERGENTES[idx % len(COLOR_DIVERGENTES)], dash='dash'),
            marker=dict(size=6)
        ))

fig_solucion.update_layout(
    template='plotly_dark',
    title='<b>✅ CORRECTO: Identificación de Países con Evolución Similar vs. Divergente</b><br><sub>Verdes: Evolución similar (Nórdicos) | Magentas: Evolución divergente</sub>',
    plot_bgcolor='black',
    paper_bgcolor='black',
    yaxis=dict(
        title='Índice de Gini',
        range=[0, 50],  # ✅ Eje desde 0
        gridcolor='#333'
    ),
    xaxis=dict(title='Año', gridcolor='#333'),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.3,
        xanchor="center",
        x=0.5
    ),
    height=600
)

fig_solucion.show()

print("\n✅ Ventajas de esta visualización:")
print("   1. Colores contrastantes: Verdes para similares, Magentas para divergentes")
print("   2. Estilos de línea diferentes: Sólida para similares, punteada para divergentes")
print("   3. Eje Y desde 0: Muestra la verdadera magnitud")
print("   4. Fácil identificar qué países tienen evolución similar (nórdicos)")
print("   5. Fácil identificar qué países tienen evolución divergente (Reino Unido vs. Francia)")



✅ SOLUCIÓN: Visualización que permite identificar similitudes y diferencias



✅ Ventajas de esta visualización:
   1. Colores contrastantes: Verdes para similares, Magentas para divergentes
   2. Estilos de línea diferentes: Sólida para similares, punteada para divergentes
   3. Eje Y desde 0: Muestra la verdadera magnitud
   4. Fácil identificar qué países tienen evolución similar (nórdicos)
   5. Fácil identificar qué países tienen evolución divergente (Reino Unido vs. Francia)


In [226]:
# ==========================================================
# ⚠️ EJERCICIO 2 - PROBLEMA: Solo Gini sin contexto adicional
# ==========================================================
print("⚠️ EJERCICIO 2: Problema de interpretación limitada")
print("   Mostrar solo el Gini sin contexto adicional")

# Seleccionar países que pueden tener Gini similar pero contextos diferentes
# Por ejemplo: Bulgaria y Francia pueden tener Gini similar pero contextos muy diferentes
PAISES_CONTEXTO = ['Bulgaria', 'Francia', 'Rumania', 'España']

df_contexto = df_paises_europeos[df_paises_europeos['Country Name'].isin(PAISES_CONTEXTO)].copy()

# Obtener el año más reciente con datos para cada país
ultimo_año_por_pais = df_contexto.groupby('Country Name')['Year'].max().reset_index()
df_ultimo = df_contexto.merge(ultimo_año_por_pais, on=['Country Name', 'Year'])

print("\n📊 Datos del último año disponible por país:")
print(df_ultimo[['Country Name', 'Year', 'Value']].to_string(index=False))

# ⚠️ PROBLEMA: Visualización que muestra SOLO el Gini, sin contexto
fig_sin_contexto = px.bar(
    df_ultimo,
    x='Country Name',
    y='Value',
    color='Country Name',
    title='<b>⚠️ PROBLEMÁTICO: Solo Gini sin Contexto Adicional</b><br><sub>Interpretación limitada: No muestra diferencias en ingresos, PIB, etc.</sub>',
    labels={'Value': 'Índice de Gini', 'Country Name': 'País'},
    color_discrete_sequence=['#808080', '#909090', '#A0A0A0', '#B0B0B0']  # Colores similares
)

fig_sin_contexto.update_layout(
    template='plotly_dark',
    plot_bgcolor='black',
    paper_bgcolor='black',
    yaxis=dict(
        title='Índice de Gini',
        range=[df_ultimo['Value'].min() - 2, df_ultimo['Value'].max() + 2],  # ⚠️ Truncado
        gridcolor='#333'
    ),
    xaxis=dict(title='País', gridcolor='#333'),
    showlegend=False
)

fig_sin_contexto.show()

print("\n⚠️ Problemas identificados:")
print("   1. Solo muestra el Gini - no hay contexto sobre ingresos, PIB, etc.")
print("   2. No se puede entender por qué dos países con Gini similar son diferentes")
print("   3. Interpretación limitada: 'Bulgaria y Francia tienen Gini similar'")
print("   4. Falta información sobre: PIB per cápita, ingresos promedio, etc.")
print("   5. Eje Y truncado exagera diferencias pequeñas")


⚠️ EJERCICIO 2: Problema de interpretación limitada
   Mostrar solo el Gini sin contexto adicional

📊 Datos del último año disponible por país:
Country Name  Year  Value
    Bulgaria  2023   39.5
      España  2023   33.4
     Francia  2023   31.8
     Rumania  2023   29.8



⚠️ Problemas identificados:
   1. Solo muestra el Gini - no hay contexto sobre ingresos, PIB, etc.
   2. No se puede entender por qué dos países con Gini similar son diferentes
   3. Interpretación limitada: 'Bulgaria y Francia tienen Gini similar'
   4. Falta información sobre: PIB per cápita, ingresos promedio, etc.
   5. Eje Y truncado exagera diferencias pequeñas


In [227]:
# ==========================================================
# ✅ EJERCICIO 2 - SOLUCIÓN: Visualización con contexto adicional
# ==========================================================
print("\n✅ SOLUCIÓN: Visualización que muestra contexto adicional")
print("   Usando la evolución temporal como contexto (ya que no tenemos datos de ingresos)")

# La solución es mostrar la EVOLUCIÓN TEMPORAL como contexto
# Esto revela que países con Gini similar pueden tener trayectorias muy diferentes

fig_con_contexto = go.Figure()

# Colores contrastantes para cada país
colores_paises = {
    'Bulgaria': '#FF6B6B',  # Rojo
    'Francia': '#0000FF',   # Azul
    'Rumania': '#FFE66D',   # Amarillo
    'España': '#95E1D3'     # Verde claro
}

for pais in PAISES_CONTEXTO:
    df_pais = df_contexto[df_contexto['Country Name'] == pais].sort_values('Year')
    if not df_pais.empty:
        # Obtener el valor más reciente para anotación
        valor_reciente = df_pais.iloc[-1]['Value']
        año_reciente = df_pais.iloc[-1]['Year']
        
        fig_con_contexto.add_trace(go.Scatter(
            x=df_pais['Year'],
            y=df_pais['Value'],
            mode='lines+markers',
            name=pais,
            line=dict(width=3, color=colores_paises.get(pais, '#FFFFFF')),
            marker=dict(size=8),
            hovertemplate=f'<b>{pais}</b><br>Año: %{{x}}<br>Gini: %{{y:.1f}}<extra></extra>'
        ))
        
        # Anotación con el valor más reciente
        fig_con_contexto.add_annotation(
            x=año_reciente,
            y=valor_reciente,
            text=f"{pais}<br>Gini: {valor_reciente:.1f}",
            showarrow=True,
            arrowhead=2,
            arrowcolor=colores_paises.get(pais, '#FFFFFF'),
            font=dict(color='white', size=10),
            bgcolor='rgba(0,0,0,0.7)',
            bordercolor=colores_paises.get(pais, '#FFFFFF'),
            borderwidth=1
        )

fig_con_contexto.update_layout(
    template='plotly_dark',
    title='<b>✅ CORRECTO: Gini con Contexto Temporal</b><br><sub>Muestra que países con Gini similar pueden tener trayectorias históricas muy diferentes</sub>',
    plot_bgcolor='black',
    paper_bgcolor='black',
    yaxis=dict(
        title='Índice de Gini',
        range=[0, 50],  # ✅ Eje desde 0
        gridcolor='#333'
    ),
    xaxis=dict(title='Año', gridcolor='#333'),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.3,
        xanchor="center",
        x=0.5
    ),
    height=600
)

fig_con_contexto.show()

print("\n✅ Ventajas de esta visualización:")
print("   1. Muestra la EVOLUCIÓN TEMPORAL como contexto adicional")
print("   2. Revela que países con Gini similar pueden tener trayectorias diferentes")
print("   3. Permite entender el CONTEXTO HISTÓRICO de cada país")
print("   4. Ejemplo: Bulgaria puede tener Gini similar a Francia, pero:")
print("      - Bulgaria: Evolución desde niveles muy altos (post-comunismo)")
print("      - Francia: Evolución más estable (país desarrollado)")
print("   5. Eje Y desde 0 muestra la verdadera magnitud")
print("\n💡 Nota: Si tuviéramos datos de ingresos/PIB, podríamos agregar un segundo eje Y")
print("   o un gráfico de dispersión Gini vs. PIB per cápita para más contexto")



✅ SOLUCIÓN: Visualización que muestra contexto adicional
   Usando la evolución temporal como contexto (ya que no tenemos datos de ingresos)



✅ Ventajas de esta visualización:
   1. Muestra la EVOLUCIÓN TEMPORAL como contexto adicional
   2. Revela que países con Gini similar pueden tener trayectorias diferentes
   3. Permite entender el CONTEXTO HISTÓRICO de cada país
   4. Ejemplo: Bulgaria puede tener Gini similar a Francia, pero:
      - Bulgaria: Evolución desde niveles muy altos (post-comunismo)
      - Francia: Evolución más estable (país desarrollado)
   5. Eje Y desde 0 muestra la verdadera magnitud

💡 Nota: Si tuviéramos datos de ingresos/PIB, podríamos agregar un segundo eje Y
   o un gráfico de dispersión Gini vs. PIB per cápita para más contexto


In [228]:
# ==========================================================
# EJERCICIO 2 - SOLUCIÓN ALTERNATIVA: Gráfico de dispersión con múltiples dimensiones
# ==========================================================
print("\n✅ SOLUCIÓN ALTERNATIVA: Visualización multi-dimensional")
print("   Mostrando Gini actual vs. Gini histórico (rango) como contexto")

# Calcular estadísticas por país para mostrar múltiples dimensiones
stats_paises = df_contexto.groupby('Country Name').agg({
    'Value': ['mean', 'min', 'max', 'std', 'count']
}).reset_index()

stats_paises.columns = ['Country Name', 'Gini_Promedio', 'Gini_Min', 'Gini_Max', 'Gini_Std', 'Num_Datos']

# Obtener el valor más reciente
ultimos_valores = df_contexto.groupby('Country Name').apply(
    lambda x: x.loc[x['Year'].idxmax(), 'Value']
).reset_index(name='Gini_Actual')

stats_paises = stats_paises.merge(ultimos_valores, on='Country Name')

# Calcular el rango histórico (diferencia entre max y min)
stats_paises['Rango_Historico'] = stats_paises['Gini_Max'] - stats_paises['Gini_Min']

print("\n📊 Estadísticas por país (contexto adicional):")
print(stats_paises[['Country Name', 'Gini_Actual', 'Gini_Promedio', 'Rango_Historico', 'Gini_Std']].to_string(index=False))

# Visualización: Gini Actual vs. Rango Histórico (muestra variabilidad)
fig_multidim = px.scatter(
    stats_paises,
    x='Gini_Actual',
    y='Rango_Historico',
    size='Gini_Std',
    color='Country Name',
    text='Country Name',
    title='<b>✅ CORRECTO: Gini Actual vs. Variabilidad Histórica (Contexto Adicional)</b><br><sub>Tamaño = Desviación estándar | Muestra que países con Gini similar pueden tener historias muy diferentes</sub>',
    labels={
        'Gini_Actual': 'Gini Actual (Último año disponible)',
        'Rango_Historico': 'Rango Histórico (Max - Min)',
        'Gini_Std': 'Desviación Estándar'
    },
    color_discrete_sequence=['#FF6B6B', '#0000FF', '#FFE66D', '#95E1D3']
)

fig_multidim.update_traces(
    textposition="top center",
    textfont=dict(color='white', size=12)
)

fig_multidim.update_layout(
    template='plotly_dark',
    plot_bgcolor='black',
    paper_bgcolor='black',
    yaxis=dict(gridcolor='#333'),
    xaxis=dict(gridcolor='#333'),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.3,
        xanchor="center",
        x=0.5
    ),
    height=600
)

fig_multidim.show()

print("\n✅ Esta visualización muestra:")
print("   1. Gini Actual (eje X): Valor más reciente")
print("   2. Rango Histórico (eje Y): Variabilidad a lo largo del tiempo")
print("   3. Tamaño del punto: Desviación estándar (estabilidad)")
print("   4. Permite identificar: Países con Gini similar pero historias diferentes")
print("   5. Ejemplo: Dos países pueden tener Gini=32, pero uno con rango 5 y otro con rango 15")



✅ SOLUCIÓN ALTERNATIVA: Visualización multi-dimensional
   Mostrando Gini actual vs. Gini histórico (rango) como contexto

📊 Estadísticas por país (contexto adicional):
Country Name  Gini_Actual  Gini_Promedio  Rango_Historico  Gini_Std
    Bulgaria         39.5      35.072000             17.9  5.091293
      España         33.4      34.562500              4.7  1.200202
     Francia         31.8      32.333333              7.4  1.466643
     Rumania         29.8      32.748387             16.3  3.823687


/var/folders/gs/dv90jgzd6_n0v0q_95xjkqwr0000gn/T/ipykernel_31407/986626807.py:15: FutureWarning:

DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.




✅ Esta visualización muestra:
   1. Gini Actual (eje X): Valor más reciente
   2. Rango Histórico (eje Y): Variabilidad a lo largo del tiempo
   3. Tamaño del punto: Desviación estándar (estabilidad)
   4. Permite identificar: Países con Gini similar pero historias diferentes
   5. Ejemplo: Dos países pueden tener Gini=32, pero uno con rango 5 y otro con rango 15


## Parte 3: Uso del Gini para la propuesta de soluciones

### Ejercicio 3: Análisis para políticas públicas

**Objetivo**: Proponer políticas públicas que podrían reducir la desigualdad, basándose en el análisis del Gini.

**Problema a identificar**: Gráfico con el eje Y truncado para exagerar los efectos de las políticas, lo cual puede dar una impresión incorrecta.

### Problemas Identificados en las Visualizaciones

1. **Parte 1 - Escalas mal ajustadas**: Eje Y truncado que exagera diferencias
2. **Parte 1 - Paleta inadecuada**: Colores similares que dificultan comparación
3. **Parte 2 - Ejercicio 1**: Líneas superpuestas con colores similares
4. **Parte 2 - Ejercicio 2**: Solo Gini sin contexto adicional

### Soluciones Implementadas

1. **Eje Y desde 0**: Muestra la verdadera magnitud de las diferencias
2. **Paleta contrastante**: Facilita distinguir entre países
3. **Agrupación visual**: Colores y estilos diferentes para grupos similares/divergentes
4. **Contexto temporal**: Evolución histórica como contexto adicional
5. **Visualización multi-dimensional**: Múltiples métricas para contexto completo


In [229]:
# ==========================================================
# ⚠️ EJERCICIO 3 - PROBLEMA: Eje Y truncado para exagerar efectos de políticas
# ==========================================================
print("⚠️ EJERCICIO 3: Problema de eje Y truncado en análisis de políticas")
print("   Esto exagera falsamente el impacto de las políticas públicas")

# Seleccionar algunos países para el análisis de políticas
PAISES_POLITICAS = ['Reino Unido', 'Francia', 'Italia', 'España']

df_politicas = df_paises_europeos[df_paises_europeos['Country Name'].isin(PAISES_POLITICAS)].copy()

# Obtener el valor más reciente de Gini para cada país
ultimo_año_pol = df_politicas.groupby('Country Name')['Year'].max().reset_index()
df_actual_pol = df_politicas.merge(ultimo_año_pol, on=['Country Name', 'Year'])

# Simular el impacto de políticas públicas (reducción del 5% en el Gini)
df_actual_pol['Gini_Proyectado'] = df_actual_pol['Value'] * 0.95
df_actual_pol['Reduccion'] = df_actual_pol['Value'] - df_actual_pol['Gini_Proyectado']

print("\n📊 Impacto proyectado de políticas públicas (reducción del 5%):")
print(df_actual_pol[['Country Name', 'Value', 'Gini_Proyectado', 'Reduccion']].to_string(index=False))

# ⚠️ PROBLEMA: Visualización con eje Y truncado que exagera el impacto
gini_min_pol = df_actual_pol['Gini_Proyectado'].min()
gini_max_pol = df_actual_pol['Value'].max()

# Crear datos para el gráfico de barras agrupadas
import numpy as np

fig_politicas_problema = go.Figure()

# Barras de Gini Actual
fig_politicas_problema.add_trace(go.Bar(
    x=df_actual_pol['Country Name'],
    y=df_actual_pol['Value'],
    name='Gini Actual',
    marker_color='#FF6B6B',
    text=[f'{val:.1f}' for val in df_actual_pol['Value']],
    textposition='auto'
))

# Barras de Gini Proyectado
fig_politicas_problema.add_trace(go.Bar(
    x=df_actual_pol['Country Name'],
    y=df_actual_pol['Gini_Proyectado'],
    name='Gini Proyectado (Políticas)',
    marker_color='#4ECDC4',
    text=[f'{val:.1f}' for val in df_actual_pol['Gini_Proyectado']],
    textposition='auto'
))

# ⚠️ PROBLEMA: Eje Y truncado para exagerar el impacto
fig_politicas_problema.update_layout(
    template='plotly_dark',
    title='<b>⚠️ PROBLEMÁTICO: Impacto de Políticas con Eje Y Truncado</b><br><sub>Exagera falsamente el efecto de las políticas (parece una reducción enorme)</sub>',
    plot_bgcolor='black',
    paper_bgcolor='black',
    barmode='group',
    yaxis=dict(
        title='Índice de Gini',
        range=[gini_min_pol - 0.5, gini_max_pol + 0.5],  # ⚠️ TRUNCADO - no desde 0
        gridcolor='#333'
    ),
    xaxis=dict(title='País', gridcolor='#333'),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.3,
        xanchor="center",
        x=0.5
    )
)

fig_politicas_problema.show()

print(f"\n⚠️ Problemas identificados:")
print(f"   1. Eje Y truncado: [{gini_min_pol - 0.5:.1f}, {gini_max_pol + 0.5:.1f}]")
print(f"   2. Debería ser [0, 50] para mostrar la verdadera magnitud")
print(f"   3. Una reducción de ~1.5 puntos parece enorme en este gráfico")
print(f"   4. En realidad, es solo una reducción del 5% (relativamente pequeña)")
print(f"   5. Esto puede llevar a conclusiones incorrectas sobre la efectividad de las políticas")


⚠️ EJERCICIO 3: Problema de eje Y truncado en análisis de políticas
   Esto exagera falsamente el impacto de las políticas públicas

📊 Impacto proyectado de políticas públicas (reducción del 5%):
Country Name  Value  Gini_Proyectado  Reduccion
 Reino Unido   32.4           30.780      1.620
      España   33.4           31.730      1.670
     Francia   31.8           30.210      1.590
      Italia   34.3           32.585      1.715



⚠️ Problemas identificados:
   1. Eje Y truncado: [29.7, 34.8]
   2. Debería ser [0, 50] para mostrar la verdadera magnitud
   3. Una reducción de ~1.5 puntos parece enorme en este gráfico
   4. En realidad, es solo una reducción del 5% (relativamente pequeña)
   5. Esto puede llevar a conclusiones incorrectas sobre la efectividad de las políticas


In [230]:
# ==========================================================
# ✅ EJERCICIO 3 - SOLUCIÓN: Visualización honesta del impacto de políticas
# ==========================================================
print("\n✅ SOLUCIÓN: Visualización honesta con eje Y desde 0")
print("   Muestra el verdadero impacto de las políticas públicas")

fig_politicas_correcta = go.Figure()

# Barras de Gini Actual
fig_politicas_correcta.add_trace(go.Bar(
    x=df_actual_pol['Country Name'],
    y=df_actual_pol['Value'],
    name='Gini Actual',
    marker_color='#FF6B6B',
    text=[f'{val:.1f}' for val in df_actual_pol['Value']],
    textposition='auto'
))

# Barras de Gini Proyectado
fig_politicas_correcta.add_trace(go.Bar(
    x=df_actual_pol['Country Name'],
    y=df_actual_pol['Gini_Proyectado'],
    name='Gini Proyectado (Políticas)',
    marker_color='#4ECDC4',
    text=[f'{val:.1f}' for val in df_actual_pol['Gini_Proyectado']],
    textposition='auto'
))

# ✅ CORRECTO: Eje Y desde 0
fig_politicas_correcta.update_layout(
    template='plotly_dark',
    title='<b>✅ CORRECTO: Impacto Real de Políticas Públicas</b><br><sub>Eje Y desde 0 muestra la verdadera magnitud del impacto (reducción del 5%)</sub>',
    plot_bgcolor='black',
    paper_bgcolor='black',
    barmode='group',
    yaxis=dict(
        title='Índice de Gini',
        range=[0, 50],  # ✅ Desde 0 - muestra la verdadera magnitud
        gridcolor='#333'
    ),
    xaxis=dict(title='País', gridcolor='#333'),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.3,
        xanchor="center",
        x=0.5
    )
)

fig_politicas_correcta.show()

print("\n✅ Ventajas de esta visualización:")
print("   1. Eje Y desde 0: Muestra la verdadera magnitud del impacto")
print("   2. Honestidad visual: Una reducción del 5% se ve como lo que es (modesta)")
print("   3. Permite evaluar correctamente la efectividad de las políticas")
print("   4. Evita conclusiones incorrectas sobre el impacto")
print("   5. Proporciona contexto real: el cambio es pequeño pero significativo")



✅ SOLUCIÓN: Visualización honesta con eje Y desde 0
   Muestra el verdadero impacto de las políticas públicas



✅ Ventajas de esta visualización:
   1. Eje Y desde 0: Muestra la verdadera magnitud del impacto
   2. Honestidad visual: Una reducción del 5% se ve como lo que es (modesta)
   3. Permite evaluar correctamente la efectividad de las políticas
   4. Evita conclusiones incorrectas sobre el impacto
   5. Proporciona contexto real: el cambio es pequeño pero significativo


### Ejercicio 4: Análisis del impacto de la crisis económica en Gini

**Objetivo**: Analizar cómo la crisis económica (por ejemplo, la crisis del COVID-19) ha impactado en la desigualdad.

**Problema a identificar**: No proporcionar contexto histórico, lo cual puede llevar a una interpretación incorrecta de las causas de las variaciones en el Gini.


In [231]:
# ==========================================================
# ⚠️ EJERCICIO 4 - PROBLEMA: Sin contexto histórico de la crisis
# ==========================================================
print("⚠️ EJERCICIO 4: Problema de falta de contexto histórico")
print("   Sin contexto, es imposible entender las causas de las variaciones")

# Seleccionar países para analizar el impacto de la crisis
PAISES_CRISIS = ['Italia', 'Reino Unido', 'Francia', 'España']

df_crisis = df_paises_europeos[df_paises_europeos['Country Name'].isin(PAISES_CRISIS)].copy()

# Filtrar solo años recientes (2018-2023) para mostrar el problema
df_crisis_reciente = df_crisis[df_crisis['Year'] >= 2018].copy()

print("\n📊 Datos recientes (2018-2023) sin contexto histórico:")
print(df_crisis_reciente.pivot_table(index='Year', columns='Country Name', values='Value').round(1))

# ⚠️ PROBLEMA: Visualización SIN contexto histórico de la crisis
fig_sin_contexto_crisis = px.line(
    df_crisis_reciente.sort_values('Year'),
    x='Year',
    y='Value',
    color='Country Name',
    markers=True,
    title='<b>⚠️ PROBLEMÁTICO: Evolución del Gini SIN Contexto Histórico</b><br><sub>No muestra la crisis COVID-19 ni el contexto previo - interpretación incorrecta</sub>',
    labels={'Value': 'Índice de Gini', 'Year': 'Año'},
    color_discrete_sequence=['#FF6B6B', '#4ECDC4', '#FFE66D', '#95E1D3']
)

fig_sin_contexto_crisis.update_layout(
    template='plotly_dark',
    plot_bgcolor='black',
    paper_bgcolor='black',
    yaxis=dict(
        title='Índice de Gini',
        range=[df_crisis_reciente['Value'].min() - 1, df_crisis_reciente['Value'].max() + 1],  # ⚠️ Truncado
        gridcolor='#333'
    ),
    xaxis=dict(title='Año', gridcolor='#333'),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.3,
        xanchor="center",
        x=0.5
    )
)

fig_sin_contexto_crisis.show()

print("\n⚠️ Problemas identificados:")
print("   1. Solo muestra años recientes (2018-2023) - sin contexto histórico")
print("   2. No indica cuándo ocurrió la crisis del COVID-19 (2020-2021)")
print("   3. No muestra la tendencia previa a la crisis")
print("   4. Imposible distinguir si el cambio es por la crisis o por tendencia previa")
print("   5. Puede llevar a interpretación incorrecta: '¿El aumento es por mala gestión?'")
print("   6. Eje Y truncado exagera las variaciones")


⚠️ EJERCICIO 4: Problema de falta de contexto histórico
   Sin contexto, es imposible entender las causas de las variaciones

📊 Datos recientes (2018-2023) sin contexto histórico:
Country Name  España  Francia  Italia  Reino Unido
Year                                              
2018            34.7     32.4    35.2         33.7
2019            34.3     31.2    34.6         32.8
2020            34.9     30.7    35.2         32.6
2021            33.9     31.5    34.8         32.4
2022            33.6     31.2    33.7          NaN
2023            33.4     31.8    34.3          NaN



⚠️ Problemas identificados:
   1. Solo muestra años recientes (2018-2023) - sin contexto histórico
   2. No indica cuándo ocurrió la crisis del COVID-19 (2020-2021)
   3. No muestra la tendencia previa a la crisis
   4. Imposible distinguir si el cambio es por la crisis o por tendencia previa
   5. Puede llevar a interpretación incorrecta: '¿El aumento es por mala gestión?'
   6. Eje Y truncado exagera las variaciones


In [232]:
# ==========================================================
# ✅ EJERCICIO 4 - SOLUCIÓN: Visualización con contexto histórico completo
# ==========================================================
print("\n✅ SOLUCIÓN: Visualización con contexto histórico y marcadores de crisis")
print("   Permite entender las causas reales de las variaciones")

# Expandir el rango temporal para incluir contexto histórico (desde 2000)
df_crisis_completo = df_crisis[df_crisis['Year'] >= 2000].copy()

fig_con_contexto_crisis = go.Figure()

# Colores para cada país
colores_crisis = {
    'Italia': '#FF6B6B',
    'Reino Unido': '#4ECDC4',
    'Francia': '#FFE66D',
    'España': '#95E1D3'
}

# Agregar líneas para cada país
for pais in PAISES_CRISIS:
    df_pais = df_crisis_completo[df_crisis_completo['Country Name'] == pais].sort_values('Year')
    if not df_pais.empty:
        fig_con_contexto_crisis.add_trace(go.Scatter(
            x=df_pais['Year'],
            y=df_pais['Value'],
            mode='lines+markers',
            name=pais,
            line=dict(width=3, color=colores_crisis.get(pais, '#FFFFFF')),
            marker=dict(size=6)
        ))

# ✅ AGREGAR CONTEXTO: Marcar el período de la crisis COVID-19
fig_con_contexto_crisis.add_vrect(
    x0=2019.8,
    x1=2021.2,
    fillcolor="rgba(255, 0, 0, 0.2)",  # Rojo semitransparente
    layer="below",
    line_width=0,
    annotation_text="CRISIS COVID-19",
    annotation_position="top left",
    annotation_font=dict(size=14, color="white", family="Arial Black"),
    annotation_bgcolor="rgba(255, 0, 0, 0.7)",
    annotation_bordercolor="white",
    annotation_borderwidth=2
)

# ✅ AGREGAR CONTEXTO: Marcar el período pre-crisis para comparación
fig_con_contexto_crisis.add_vrect(
    x0=2015,
    x1=2019.8,
    fillcolor="rgba(0, 255, 0, 0.1)",  # Verde muy transparente
    layer="below",
    line_width=0,
    annotation_text="Pre-Crisis",
    annotation_position="top right",
    annotation_font=dict(size=10, color="white"),
    annotation_bgcolor="rgba(0, 255, 0, 0.3)"
)

fig_con_contexto_crisis.update_layout(
    template='plotly_dark',
    title='<b>✅ CORRECTO: Impacto de la Crisis COVID-19 con Contexto Histórico</b><br><sub>Muestra la tendencia previa, la crisis (2020-2021) y la recuperación posterior</sub>',
    plot_bgcolor='black',
    paper_bgcolor='black',
    yaxis=dict(
        title='Índice de Gini',
        range=[0, 50],  # ✅ Eje desde 0
        gridcolor='#333'
    ),
    xaxis=dict(title='Año', gridcolor='#333'),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.3,
        xanchor="center",
        x=0.5
    ),
    height=600
)

fig_con_contexto_crisis.show()

print("\n✅ Ventajas de esta visualización:")
print("   1. Contexto histórico completo: Muestra tendencias desde 2000")
print("   2. Marcador de crisis: Zona sombreada roja indica el período COVID-19")
print("   3. Zona pre-crisis: Permite comparar con el período anterior")
print("   4. Permite identificar:")
print("      - Tendencias previas a la crisis")
print("      - Impacto inmediato de la crisis (2020-2021)")
print("      - Recuperación post-crisis (2022-2023)")
print("   5. Evita interpretación incorrecta: 'El aumento es por la crisis, no por mala gestión'")
print("   6. Eje Y desde 0: Muestra la verdadera magnitud de los cambios")



✅ SOLUCIÓN: Visualización con contexto histórico y marcadores de crisis
   Permite entender las causas reales de las variaciones



✅ Ventajas de esta visualización:
   1. Contexto histórico completo: Muestra tendencias desde 2000
   2. Marcador de crisis: Zona sombreada roja indica el período COVID-19
   3. Zona pre-crisis: Permite comparar con el período anterior
   4. Permite identificar:
      - Tendencias previas a la crisis
      - Impacto inmediato de la crisis (2020-2021)
      - Recuperación post-crisis (2022-2023)
   5. Evita interpretación incorrecta: 'El aumento es por la crisis, no por mala gestión'
   6. Eje Y desde 0: Muestra la verdadera magnitud de los cambios


In [233]:
# ==========================================================
# EJERCICIO 4 - ANÁLISIS DETALLADO: Comparación Pre-Crisis vs. Durante Crisis
# ==========================================================
print("\n📊 Análisis detallado: Comparación Pre-Crisis vs. Durante Crisis")

# Calcular promedios por período
df_crisis_completo['Periodo'] = df_crisis_completo['Year'].apply(
    lambda x: 'Pre-Crisis (2015-2019)' if 2015 <= x < 2020 
    else 'Crisis (2020-2021)' if 2020 <= x <= 2021 
    else 'Post-Crisis (2022-2023)' if x >= 2022 
    else 'Histórico (2000-2014)'
)

# Estadísticas por período y país
stats_crisis = df_crisis_completo.groupby(['Country Name', 'Periodo'])['Value'].agg(['mean', 'std', 'count']).reset_index()
stats_crisis.columns = ['País', 'Periodo', 'Gini_Promedio', 'Desviacion', 'Num_Datos']

print("\n📊 Estadísticas por período:")
print(stats_crisis.pivot_table(index='País', columns='Periodo', values='Gini_Promedio').round(2))

# Visualización de barras comparando períodos
fig_comparacion_periodos = px.bar(
    stats_crisis[stats_crisis['Periodo'].isin(['Pre-Crisis (2015-2019)', 'Crisis (2020-2021)', 'Post-Crisis (2022-2023)'])],
    x='País',
    y='Gini_Promedio',
    color='Periodo',
    barmode='group',
    title='<b>✅ Comparación: Pre-Crisis vs. Crisis vs. Post-Crisis</b><br><sub>Muestra el impacto real de la crisis COVID-19 en cada país</sub>',
    labels={'Gini_Promedio': 'Gini Promedio', 'País': 'País'},
    color_discrete_map={
        'Pre-Crisis (2015-2019)': '#4ECDC4',  # Turquesa
        'Crisis (2020-2021)': '#FF6B6B',       # Rojo
        'Post-Crisis (2022-2023)': '#FFE66D'   # Amarillo
    }
)

fig_comparacion_periodos.update_layout(
    template='plotly_dark',
    plot_bgcolor='black',
    paper_bgcolor='black',
    yaxis=dict(
        title='Gini Promedio',
        range=[0, 50],  # ✅ Desde 0
        gridcolor='#333'
    ),
    xaxis=dict(title='País', gridcolor='#333'),
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.3,
        xanchor="center",
        x=0.5
    )
)

fig_comparacion_periodos.show()

print("\n✅ Esta visualización permite:")
print("   1. Comparar directamente el Gini antes, durante y después de la crisis")
print("   2. Identificar qué países fueron más afectados")
print("   3. Ver qué países se recuperaron más rápido")
print("   4. Entender el impacto real de la crisis en cada contexto nacional")



📊 Análisis detallado: Comparación Pre-Crisis vs. Durante Crisis

📊 Estadísticas por período:
Periodo      Crisis (2020-2021)  Histórico (2000-2014)  \
País                                                     
España                     34.4                  34.38   
Francia                    31.1                  32.09   
Italia                     35.0                  34.41   
Reino Unido                32.5                  34.85   

Periodo      Post-Crisis (2022-2023)  Pre-Crisis (2015-2019)  
País                                                          
España                          33.5                   35.14  
Francia                         31.5                   31.96  
Italia                          34.0                   35.26  
Reino Unido                      NaN                   33.10  



✅ Esta visualización permite:
   1. Comparar directamente el Gini antes, durante y después de la crisis
   2. Identificar qué países fueron más afectados
   3. Ver qué países se recuperaron más rápido
   4. Entender el impacto real de la crisis en cada contexto nacional


## Resumen Final: Problemas y Soluciones


### Problemas Identificados en las Visualizaciones

1. **Parte 1 - Escalas mal ajustadas**: Eje Y truncado que exagera diferencias
2. **Parte 1 - Paleta inadecuada**: Colores similares que dificultan comparación
3. **Parte 2 - Ejercicio 1**: Líneas superpuestas con colores similares
4. **Parte 2 - Ejercicio 2**: Solo Gini sin contexto adicional
5. **Parte 3 - Ejercicio 3**: Eje Y truncado para exagerar efectos de políticas
6. **Parte 3 - Ejercicio 4**: Falta de contexto histórico para interpretar crisis

### Soluciones Implementadas

1. **Eje Y desde 0**: Muestra la verdadera magnitud de las diferencias
2. **Paleta contrastante**: Facilita distinguir entre países
3. **Agrupación visual**: Colores y estilos diferentes para grupos similares/divergentes
4. **Contexto temporal**: Evolución histórica como contexto adicional
5. **Visualización multi-dimensional**: Múltiples métricas para contexto completo
6. **Marcadores de eventos**: Zonas sombreadas para indicar períodos de crisis
7. **Comparación de períodos**: Análisis antes/durante/después de eventos importantes
